In [1]:
import pandas as pd
import numpy as np
#一 数据收集 start 
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [2]:
#二 数据预处理 start 
print(train['Attrition'].value_counts())
train['Attrition']=train['Attrition'].map(lambda x:1 if x=='Yes' else 0)

No     988
Yes    188
Name: Attrition, dtype: int64


In [3]:
#train.isna().sum()#查看是否有空值 Step1，对数据进行探索 
#Step2，去掉无用特征，处理分类特征
train= train.drop(['EmployeeNumber', 'StandardHours'], axis=1)
test=test.drop(['EmployeeNumber', 'StandardHours'], axis=1)

In [4]:
#分类特征进行特征值编码：
from sklearn import preprocessing 

attr=['Age','BusinessTravel','Department','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']
lbe_list=[]
for fea in attr:
    lbe=preprocessing.LabelEncoder()
    train[fea]=lbe.fit_transform(train[fea])
    test[fea]=lbe.transform(test[fea])
    lbe_list.append(lbe)
print(lbe_list)

[LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder()]


In [5]:
from tpot import TPOTClassifier

In [6]:
#Step4，模型训练，得出预测结果

from sklearn.model_selection import  train_test_split
from sklearn.metrics import accuracy_score
X_train,X_valid,Y_train,Y_valid=train_test_split(train.drop('Attrition',axis=1),train['Attrition'],test_size=0.2,random_state=42)
tpot=TPOTClassifier(generations=5,population_size=20,verbosity=2)
tpot.fit(X_train,Y_train)
print(tpot.score(X_valid,Y_valid))
#predict=model.predict(test)
#predict_y=model.predict(X_valid)
#print('LR准确率: %0.4lf' % accuracy_score(predic_y, Y_valid))
#print(predict)
#test['Attrition']=predict
#test['Attrition']=test['Attrition'].map(lambda x:1 if x>=0.5 else 0)
#test['Attrition']=predict
#test[['user_id','Attrition']].to_csv('Submission.csv')

Generation 1 - Current best internal CV score: 0.8691489361702128
Generation 2 - Current best internal CV score: 0.876595744680851
Generation 3 - Current best internal CV score: 0.876595744680851
Generation 4 - Current best internal CV score: 0.876595744680851
Generation 5 - Current best internal CV score: 0.8776595744680851

Best pipeline: ExtraTreesClassifier(GaussianNB(input_matrix), bootstrap=True, criterion=gini, max_features=0.45, min_samples_leaf=8, min_samples_split=20, n_estimators=100)
0.8389830508474576


In [15]:
predict=tpot.predict_proba(test)[:, 1]
test['Attrition']=predict
#test['Attrition']=test['Attrition'].map(lambda x:1 if x>=0.5 else 0)
#test['Attrition']=predict
test[['user_id','Attrition']].to_csv('Submission.csv')